In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
from loguru import logger

from utils.cri import calc_cpa, calc_cri

In [11]:
logger.add("../logs/risk-assessment-{time:YYYYMMDD}.log", rotation="00:00")

3

In [12]:
DATA_PATH = "../data/sample_rows.csv"

logger.info("Reading data from {}", DATA_PATH)
data = pd.read_csv(DATA_PATH)

2024-11-20 13:03:56.486 | INFO     | __main__:<module>:3 - Reading data from ../data/sample_rows.csv


In [13]:
cri_values = []  # List to store CRI values
euclidean_distance = []  # List for euclidean_distance
rel_movement_direction = []  # List for rel_movement_direction
azimuth_target_to_own = []  # List for azimuth_target_to_own

for idx, row in data.iterrows():
    logger.info(f"Calculating CPA for vessel_1: {row['vessel_1']} and vessel_2: {row['vessel_2']}...")
    cpa = calc_cpa(row)

    # Extract CPA-related data
    euclidean_distance.append(cpa["euclidian_dist"][0][0])
    rel_movement_direction.append(cpa["rel_movement_direction"])
    azimuth_target_to_own.append(cpa["azimuth_target_to_own"])

    # Calculate CRI
    cri = calc_cri(
        row,
        cpa["euclidian_dist"],
        cpa["rel_movement_direction"],
        cpa["azimuth_target_to_own"],
        cpa["rel_bearing"],
        cpa["dcpa"],
        cpa["tcpa"],
        cpa["rel_speed_mag"],
    )
    logger.info(f"Calculated CRI: {cri}")
    cri_values.append(cri)

# Add the collected data to the DataFrame
data['euclidean_distance'] = euclidean_distance
data['rel_movement_direction'] = rel_movement_direction
data['azimuth_target_to_own'] = azimuth_target_to_own
data['ves_cri'] = cri_values

# Save the updated DataFrame to a CSV file
output_file = '../data/training_data.csv'  # Specify the output file path
data.to_csv(output_file, index=False)
logger.info(f"Updated data with CRI and CPA values saved to {output_file}")

2024-11-20 13:03:56.501 | INFO     | __main__:<module>:7 - Calculating CPA for vessel_1: 246447000 and vessel_2: 477035800...
2024-11-20 13:03:56.503 | INFO     | __main__:<module>:26 - Calculated CRI: 0.8452304724406478
2024-11-20 13:03:56.504 | INFO     | __main__:<module>:7 - Calculating CPA for vessel_1: 246447000 and vessel_2: 266465000...
2024-11-20 13:03:56.505 | INFO     | __main__:<module>:26 - Calculated CRI: 0.870753726137985
2024-11-20 13:03:56.510 | INFO     | __main__:<module>:38 - Updated data with CRI and CPA values saved to ../data/training_data.csv
